# Skeletonize and export data





In [9]:
!pip install SimpleITK
!pip install skan #older version
!pip install tifffile

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [11]:
import SimpleITK as sitk
from skimage.morphology import skeletonize, thin, medial_axis
from skan import skeleton_to_csgraph, Skeleton
from skan import summarize
from skan import draw
import tifffile as tiff

import matplotlib.pyplot as plt
import numpy as np
import cv2

In [12]:
def load_img_from_tiff(path2img):
    """
    Parameters
    ----------
    path2img: str 
        path to image Tiff file
    
    Returns
    -------
    img_array: np.array
        image data in numpy format
    """
    img = sitk.ReadImage(path2img)
    img_array = sitk.GetArrayFromImage(img)
    return(img_array)

Skeletonize single frame

In [ ]:
skimage_skeleton.shape

In [ ]:
single_thresh = load_img_from_tiff("/content/Test n.1")

# single_thresh = cv2.resize(single_thresh, dsize=(256,256), interpolation=cv2.INTER_CUBIC)

single_thresh = single_thresh.astype('float64')
single_thresh = (single_thresh - np.min(single_thresh))/np.ptp(single_thresh)
skimage_skeleton = skeletonize(single_thresh)
skan_skeleton = Skeleton(skimage_skeleton)
df=summarize(skan_skeleton)
df.to_csv("single_summary.csv")
df.tail()

# df.drop(df[df["branch-type"] < 2].index, inplace=True)

In [ ]:
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (10,10)
# draw.overlay_skeleton_2d(single_thresh, skimage_skeleton, dilate=1, axes=ax);
plt.imshow(single_thresh+skimage_skeleton, cmap='jet')
plt.axis("off")
plt.title("Overlayed Skeleton")

In [ ]:
#overlay nodes 
res_st = cv2.resize(single_thresh, dsize=(256,256), interpolation=cv2.INTER_CUBIC)
ys=np.array(df['image-coord-src-0'])
xs=np.array(df['image-coord-src-1'])
node_image = np.zeros([305,305], dtype=np.uint8)
# print(node_image.shape)
node_image[ys.astype(np.uint16), xs.astype(np.uint16)] = 4.
plt.imshow(node_image+skimage_skeleton+single_thresh)
plt.title("Overlayed Skeleton and Nodes")
plt.colorbar()
# plt.scatter(ys,xs)


Skeletonize multiple Z slices

In [33]:
path = "/content/drive/MyDrive/mydata/sea_urchin_data/3D/Galleria Piastra Modello 1 100x100 Echi1-10x.tif"
data = load_img_from_tiff(path)
for i in range(len(data)):
    data[i] = data[i]*(255.0/data[i].max())

In [37]:
blurred_data = np.zeros_like(data)
for i in range(len(blurred_data)):
    blurred_data[i] = cv2.GaussianBlur(data[i], (3,3),3)

thresholded_data = np.zeros_like(data)
for i in range(len(thresholded_data)):
    bin, thresholded_data[i] = cv2.threshold(blurred_data[i],175, 255, cv2.THRESH_BINARY)

medial_axis_skel = np.zeros_like(data)
for i in range(len(thresholded_data)):
    # skel, distance = medial_axis(thresholded_data[i], return_distance=True)
    # dist_on_skel = distance * skel
    # medial_axis_skel[i]=skel
    skel = skeletonize(thresholded_data[i]//255)
    medial_axis_skel[i]=skel

In [ ]:
skan_skel = Skeleton(medial_axis_skel)
df=summarize(skan_skel)
df.to_csv("z_slices_summary.csv")
#drop brances with length < 2
# df.drop(df[df["branch-distance"] < 2].index, inplace=True)
df